### s000 environment setup

In [0]:
# libraries
import os

import pyspark
from pyspark import sql
from pyspark.sql import functions as f
from pyspark.sql import Window

In [0]:
%run "./utils_spark_df"

In [0]:
%run "./utils_stratified_sampling"

In [0]:
%run "./sc_utils_stratified_sampling"

In [0]:
# ------------- Use snowflake utility
sfUtils = sc._jvm.net.snowflake.spark.snowflake.Utils

# ------------ login to snowflake
password = dbutils.secrets.get(scope = "auea-kv-sbx-dxdtlprdct01", key = "sfdbrsdskey")

options = {
  "sfUrl": "vodafonenz_prod.australia-east.azure.snowflakecomputing.com/", 
  "sfUser": "SVC_LAB_DS_DATABRICKS",
  "pem_private_key": password.replace('\\n', '\n'),
  "sfDatabase": "LAB_ML_STORE",
  "sfSchema": "SANDBOX",
  "sfWarehouse": "LAB_DS_WH"
}

In [0]:
dir_fs_data_parent = "/mnt/feature-store-prod-lab"
dir_mls_data_parent = "/mnt/ml-store-prod-lab/classification"

In [0]:
dir_mls_data_score = os.path.join(dir_mls_data_parent, "d400_model_score")

In [0]:
dir_fs_data_meta = os.path.join(dir_fs_data_parent, 'd000_meta')
dir_fs_data_raw =  os.path.join(dir_fs_data_parent, 'd100_raw')
dir_fs_data_int =  os.path.join(dir_fs_data_parent, "d200_intermediate")
dir_fs_data_prm =  os.path.join(dir_fs_data_parent, "d300_primary")
dir_fs_data_fea =  os.path.join(dir_fs_data_parent, "d400_feature")
dir_fs_data_target = os.path.join(dir_fs_data_parent, "d500_movement")
dir_fs_data_serv = os.path.join(dir_fs_data_parent, "d600_serving")

### s100 data import

In [0]:
vt_param_reporting_date = "2025-02-23"
vt_param_reporting_cycle_type = "rolling cycle"

In [0]:
df_fs_master = spark.read.format("delta").load(os.path.join(dir_fs_data_serv, "serv_mobile_oa_consumer"))
df_fs_ifp_srvc = spark.read.format("delta").load(os.path.join(dir_fs_data_fea, "d401_mobile_oa_consumer/fea_ifp_device_on_service"))
df_fs_ifp_bill = spark.read.format("delta").load(os.path.join(dir_fs_data_fea, "d401_mobile_oa_consumer/fea_ifp_device_on_bill"))
df_mvnt_ifp_srvc = spark.read.format('delta').load(os.path.join(dir_fs_data_target, "d501_mobile_oa_consumer/mvmt_ifp_upsell_on_service"))
df_mvnt_ifp_bill = spark.read.format('delta').load(os.path.join(dir_fs_data_target, "d501_mobile_oa_consumer/mvmt_ifp_upsell_on_bill"))

In [0]:
#df_mls_score_dr_apple = spark.read.format("delta").load(os.path.join(dir_mls_data_score, "mobile_oa_consumer_srvc_device_replacement_apple_pred30d"))
#df_mls_score_dr_apple = spark.read.format("delta").load(os.path.join(dir_mls_data_score, "mobile_oa_consumer_srvc_device_replacement_apple_pred30d"))
df_mls_score_dr = spark.read.format("delta").load(os.path.join(dir_mls_data_score, "mobile_oa_consumer_srvc_device_replacement_pred30d"))
df_mls_score_ifp = spark.read.format("delta").load(os.path.join(dir_mls_data_score, "mobile_oa_consumer_srvc_upsell_ifp_pred30d"))
df_mls_score_churn = spark.read.format("delta").load(os.path.join(dir_mls_data_score, "mobile_oa_consumer_srvc_churn_pred30d"))
df_mls_score_ar = spark.read.format("delta").load(os.path.join(dir_mls_data_score, "mobile_oa_consumer_srvc_writeoff_pred120d"))

In [0]:
df_output_campaign = spark.read.format("delta").load('/mnt/ml-lab/dev_users/sc/ml_campaigns/wallet/250228-JS-MOBPM-iPhone-SE-Launch')

In [0]:
query_sfmc = """ 
select  * from PROD_MAR_TECH.SERVING.SFMC_EMAIL_PERFORMANCE
where campaignname in ('250228-JS-MOBPM-iPhone-SE-Launch_part2', '250228-JS-MOBPM-iPhone-SE-Launch')
"""


df_sfmc_email= (
    spark.read
    .format("snowflake")
    .options(**options)
    .option(
        "query"
        , query_sfmc
    )
    .load()
)


In [0]:
display(
    df_sfmc_email
    .agg(
        f.count('*')
        , f.countDistinct('customer_id')
    )
)

display(df_sfmc_email
        .join(df_output_campaign, f.col('customer_id') == f.col('fs_cust_id'), 'anti')
        )

In [0]:
df_base_score_ifp = (
    df_mls_score_ifp
    .filter(f.col("reporting_date") == vt_param_reporting_date)
    .filter(f.col("reporting_cycle_type") == vt_param_reporting_cycle_type)
    .withColumn(
        'rank_acct'
        , f.row_number().over(
            Window
            .partitionBy('fs_cust_id', 'fs_acct_id')
            .orderBy(f.desc('propensity_top_ntile'))
        )
    )
    .filter(f.col('rank_acct')==1)
    .withColumn(
        'rank_cust'
        , f.row_number().over(
            Window
            .partitionBy('fs_cust_id')
            .orderBy(f.desc('propensity_top_ntile'))
        )
    )
    .filter(f.col('rank_cust')==1)
    .select(
        "fs_cust_id"
        # , "fs_acct_id"
        # , "fs_srvc_id"
        , f.col("propensity_score").alias("ifp_score")
        , f.col("propensity_segment_qt").alias("ifp_segment")
        , f.col("propensity_top_ntile").alias("ifp_top_ntile")
    )
   
)

df_base_score_dr = (
    df_mls_score_dr
    .filter(f.col("reporting_date") == vt_param_reporting_date)
    .filter(f.col("reporting_cycle_type") == vt_param_reporting_cycle_type)
    .withColumn(
        'rank_acct'
        , f.row_number().over(
            Window
            .partitionBy('fs_cust_id','fs_acct_id')
            .orderBy(f.desc('propensity_top_ntile'))
        )
    )
    .filter(f.col('rank_acct')==1)
    .withColumn(
        'rank_cust'
        , f.row_number().over(
            Window
            .partitionBy('fs_cust_id')
            .orderBy(f.desc('propensity_top_ntile'))
        )
    )
    .filter(f.col('rank_cust')==1)
    .select(
        "fs_cust_id"
        # , "fs_acct_id"
        # , "fs_srvc_id"
        , f.col("propensity_score").alias("dr_score")
        , f.col("propensity_segment_qt").alias("dr_segment")
        , f.col("propensity_top_ntile").alias("dr_top_ntile")
    )   
)

df_base_score_churn = (
    df_mls_score_churn
    .filter(f.col("reporting_date") == vt_param_reporting_date)
    .filter(f.col("reporting_cycle_type") == vt_param_reporting_cycle_type)
    .withColumn(
        'rank_acct'
        , f.row_number().over(
            Window
            .partitionBy('fs_cust_id','fs_acct_id')
            .orderBy(f.desc('propensity_top_ntile'))
        )
    )
    .filter(f.col('rank_acct')==1)
    .withColumn(
        'rank_cust'
        , f.row_number().over(
            Window
            .partitionBy('fs_cust_id')
            .orderBy(f.desc('propensity_top_ntile'))
        )
    )
    .filter(f.col('rank_cust')==1)
    .select(
        "fs_cust_id"
        # , "fs_acct_id"
        # , "fs_srvc_id"
        , f.col("propensity_score").alias("churn_score")
        , f.col("propensity_segment_qt").alias("churn_segment")
        , f.col("propensity_top_ntile").alias("churn_top_ntile")
    )
)

df_base_score_ar = (
    df_mls_score_ar
    .filter(f.col("reporting_date") == vt_param_reporting_date)
    .filter(f.col("reporting_cycle_type") == vt_param_reporting_cycle_type)
    .withColumn(
        'rank_acct'
        , f.row_number().over(
            Window
            .partitionBy('fs_cust_id','fs_acct_id')
            .orderBy(f.desc('propensity_top_ntile'))
        )
    )
    .filter(f.col('rank_acct')==1)
    .withColumn(
        'rank_cust'
        , f.row_number().over(
            Window
            .partitionBy('fs_cust_id')
            .orderBy(f.desc('propensity_top_ntile'))
        )
    )
    .filter(f.col('rank_cust')==1)
    .select(
        "fs_cust_id"
        # , "fs_acct_id"
        # , "fs_srvc_id"
        , f.col("propensity_score").alias("risk_score")
        , f.col("propensity_segment_qt").alias("risk_segment")
        , f.col("propensity_top_ntile").alias("risk_top_ntile")
    )
)

In [0]:
df_email_output=(
    df_sfmc_email.alias('a')
    .join(df_base_score_ifp.alias('c'), f.col('a.CUSTOMER_ID') == f.col('c.fs_cust_id'), 'inner')
    .join(df_base_score_dr.alias('d'), f.col('a.CUSTOMER_ID') == f.col('d.fs_cust_id'), 'inner')
    .join(df_base_score_churn.alias('e'), f.col('a.CUSTOMER_ID') == f.col('e.fs_cust_id'), 'inner')
    #.withColumn('index', f.count('*').over(Window.partitionBy('a.CUSTOMER_ID')))
    #.filter(f.col('index') >2)
    .select(
        'CUSTOMER_ID'
        , 'c.fs_cust_id'
        , 'ifp_score'
        , 'ifp_segment'
        , 'ifp_top_ntile'
        , 'dr_score'
        , 'dr_segment'
        , 'dr_top_ntile'
        , 'churn_score'
        , 'churn_segment'
        , 'churn_top_ntile'
    )
 )



In [0]:
display(
    df_email_output
    #.filter(f.col("target_segment") == 'a.target')
    .groupBy("ifp_segment")
    .agg(f.countDistinct("fs_cust_id").alias("acct"))
    .withColumn(
        "acct_tot"
        , f.sum("acct").over(Window.partitionBy(f.lit(1)))
    )
    .withColumn(
        "pct"
        , f.col("acct") / f.col("acct_tot")
    )
)

display(
    df_email_output
    #.filter(f.col("target_segment") == 'a.target')
    .groupBy("dr_segment")
    .agg(f.countDistinct("fs_cust_id").alias("acct"))
    .withColumn(
        "acct_tot"
        , f.sum("acct").over(Window.partitionBy(f.lit(1)))
    )
    .withColumn(
        "pct"
        , f.col("acct") / f.col("acct_tot")
    )
)


display(
    df_email_output
    #.filter(f.col("target_segment") == 'a.target')
    .groupBy("churn_segment")
    .agg(f.countDistinct("fs_cust_id").alias("acct"))
    .withColumn(
        "acct_tot"
        , f.sum("acct").over(Window.partitionBy(f.lit(1)))
    )
    .withColumn(
        "pct"
        , f.col("acct") / f.col("acct_tot")
    )
)



display(
    df_email_output
    .agg(
        f.countDistinct("fs_cust_id")
        , f.median("churn_score")
        , f.avg("churn_score")
        , f.median("dr_score")
        , f.avg("dr_score")
        , f.median("ifp_score")
        , f.avg("ifp_score")
    )
)

### s102 global control

In [0]:
df_gc_curr = spark.read.format("delta").load("/mnt/ml-store-dev/dev_users/dev_el/marketing_programs/global_control/mobile_oa_consumer")

df_gc_curr = (
    df_gc_curr
    .select("fs_cust_id")
    .distinct()
)

print(df_gc_curr.count())

display(
    df_gc_curr
    .limit(10)
)

## s200 Data processing

### s200 base candidate

In [0]:
df_base_full = (
    df_fs_master
    .filter(f.col("reporting_date") == vt_param_reporting_date)
    .filter(f.col("reporting_cycle_type") == vt_param_reporting_cycle_type)
)

display(
    df_base_full
    .limit(100)
)

display(
    df_base_full
    .count()
) 

### s202 exclusion flag

In [0]:
# current global control
df_tmp_excl_01 = (
    df_gc_curr
    .select("fs_cust_id")
    .distinct()
    .withColumn(
        "gc_curr_flag"
        , f.lit('Y')
    )
)


In [0]:
df_apple_cust = (
    df_base_full
    .withColumn(
        "network_dvc_brand_std"
        , f.when(
            f.lower(f.col("network_dvc_brand")).isin(["apple"])
            , f.lower(f.col("network_dvc_brand"))
        ).otherwise(
            f.lit("others")
        )
    )
    .groupBy('fs_cust_id')
    .agg(
        f.countDistinct('fs_srvc_id').alias('distinct_services'),
        f.max(f.when(f.col('network_dvc_brand_std') == 'apple', 1).otherwise(0)).alias('has_apple')
    )
)


In [0]:
df_proc_full = (
    df_base_full
    .select(
        "reporting_date"
        , "reporting_cycle_type"
        , "fs_cust_id"
    )
    .distinct()
    .join(
        df_base_score_ifp
        , ["fs_cust_id"]
        , "left"
    )
    .join(
        df_base_score_dr
        , ["fs_cust_id"]
        , "left"
    )
    .join(
        df_base_score_churn
        , ["fs_cust_id"]
        , "left"
    )
    .join(
        df_base_score_ar
        , ["fs_cust_id"]
        , "left"
    )
    .join(
        df_tmp_excl_01
        , ["fs_cust_id"]
        , "left"
    )
    # .join(
    #     df_tmp_excl_04
    #     , ["fs_cust_id"]
    #     , "left"
    # )
    .join(
        df_apple_cust
        , ['fs_cust_id']
        , 'left'
    )
    .join(df_email_output.alias('a')
          .select('fs_cust_id')
          .distinct()
          , ['fs_cust_id']
          , 'left'
          )
    .fillna(
        value='N'
        , subset=['gc_curr_flag']
    )
    .withColumn(
        "target_segment"
        , f.when(
            f.col('a.fs_cust_id').isNotNull()
           , 'a.target'
        )
        .when(
            (
                (f.col("gc_curr_flag") == 'Y')
            )
            , f.lit("z2.global control - curr")
        )
        .when(
           (
               f.col('has_apple') == 0
           )
           , f.lit('z7.Android Device')
        )
        .when(
            (
                f.col("risk_top_ntile") >= 98
            )
            , f.lit("z6.High Bad Debt Risk")
        )
        .otherwise(f.lit("a.misc"))
    )
    .withColumn("churn_top_ntile", f.ntile(20).over(Window.orderBy(f.desc("churn_score"))))
    .withColumn("dr_top_ntile", f.ntile(20).over(Window.orderBy(f.desc("dr_score"))))
    .withColumn("ifp_top_ntile", f.ntile(20).over(Window.orderBy(f.desc("ifp_score"))))
)

In [0]:
display(
    df_proc_full
    .groupBy("target_segment")
    .agg(
        f.count("*")
        , f.countDistinct("fs_cust_id")
    )
    .orderBy("target_segment")
)

display(df_proc_full.limit(100))
display(df_proc_full.count())

In [0]:
df_campaign_full = (
    df_proc_full
    .withColumn(
        "ifp_rank"
        , f.row_number().over(
            Window
            .partitionBy(f.lit(1))
            .orderBy(f.desc("ifp_score"))
        )
    )
    .withColumn(
        "dr_rank"
        , f.row_number().over(
            Window
            .partitionBy(f.lit(1))
            .orderBy(f.desc("dr_score"))
        )
    )
    .withColumn(
        "churn_rank"
        , f.row_number().over(
            Window
            .partitionBy(f.lit(1))
            .orderBy(f.desc("churn_score"))
        )
    )
)


In [0]:
(
    df_campaign_full
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    #.option("partitionOverwriteMode", "dynamic")
    .save('/mnt/ml-lab/dev_users/dev_sc/ml_campaigns/regen_control/250228-JS-MOBPM-iPhone-SE-Launch') 
)

In [0]:
df_campaign_full = spark.read.format('delta').load('/mnt/ml-lab/dev_users/dev_sc/ml_campaigns/regen_control/250228-JS-MOBPM-iPhone-SE-Launch')

### s206 local control - H

In [0]:
vt_param_control_size = 25000
ls_param_strata = [ "ifp_top_ntile", "churn_top_ntile", 'dr_top_ntile']

df_sample_target = create_sample_target(
     df_campaign_full
    .filter(f.col("target_segment") == 'a.target')
    #.filter(f.col("wallet_eligibility_flag") == 'Y')
    , ls_param_strata
)

df_campaign_cand_control = find_similar_sample(
    df_campaign_full
    .filter(f.col("target_segment") != 'a.target')
    .filter(f.col("target_segment") != 'z7.Android Device')
    # .filter(f.col("ch01_flag") != 'Y')
    # .filter(f.col("ch02_flag") != 'Y')
    # .filter(f.col("ch03_flag") != 'Y')
    , size = vt_param_control_size
    , strata = ls_param_strata
    , df_target = df_sample_target
)


print("control")
display(
    df_campaign_cand_control
    .agg(
        f.countDistinct("fs_cust_id")
        , f.median("churn_score")
        , f.avg("churn_score")
        , f.median("dr_score")
        , f.avg("dr_score")
        , f.median("ifp_score")
        , f.avg("ifp_score")
    )
)


print("target")
display(
    df_campaign_full
    .filter(f.col('target_segment') == 'a.target')
    .agg(
        f.countDistinct("fs_cust_id")
        , f.median("churn_score")
        , f.avg("churn_score")
        , f.median("dr_score")
        , f.avg("dr_score")
        , f.median("ifp_score")
        , f.avg("ifp_score")
    )
)

df_campaign_cand_target = (
    df_campaign_full
    .filter(f.col('target_segment') == 'a.target')
)

In [0]:
(
    df_campaign_cand_control
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    #.option("partitionOverwriteMode", "dynamic")
    .save('/mnt/ml-lab/dev_users/dev_sc/ml_campaigns/regen_control/250228-JS-MOBPM-iPhone-SE-Launch_control') 
)

In [0]:
(
    df_campaign_cand_target
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    #.option("partitionOverwriteMode", "dynamic")
    .save('/mnt/ml-lab/dev_users/dev_sc/ml_campaigns/regen_control/250228-JS-MOBPM-iPhone-SE-Launch_target') 
)